In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 83.2 MB/s eta 0:00:0

In [ ]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.2 MB/s eta 0:00:00
Found existing installation: transformers 4.32.0
Uninstalling transformers-4.32.0:
  Successfully uninstalled transformers-4.32.0
Found existing installation: accelerate 0.21.0
Uninstalling accelerate-0.21.0:
  Successfully uninstalled accelerate-0.21.0
  Using cached transformers-4.32.0-py3-none-any.whl (7.5 MB)
  Using cached accelerate-0.21.0-py3-none-any.whl (244 kB)


In [ ]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Set display options to show the full content of columns
pd.set_option('display.max_colwidth', None)

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
model_ckpt = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

pip install pathlib

In [ ]:
from pathlib import Path

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
base_directory=Path("/content/drive/MyDrive/Text Extraction")

In [ ]:
#Reading the data
a=pd.read_excel(f"{base_directory}/IntentData2.xlsx",sheet_name="Set 1")

In [ ]:
b=pd.read_excel(f"{base_directory}/IntentData2.xlsx",sheet_name="Set 2")

In [ ]:
df=pd.concat([a,b],axis=0)

In [ ]:
df1=df.dropna(subset=['Positive', 'Negative (Other)', 'Not Resolved',
       'Hold', 'Follow-up, Communication', 'Knowledge', 'Effort',
       'Professionalism'],how="all")

In [ ]:
df2=pd.melt(df1, id_vars=['Original Comment'], value_vars=['Positive', 'Negative (Other)', 'Not Resolved',
       'Hold', 'Follow-up, Communication', 'Knowledge', 'Effort',
       'Professionalism'],var_name="type",value_name="target")

In [ ]:
df2=df2.dropna(subset="target")

In [ ]:
df3=df2.reset_index().reset_index().drop("index",axis=1)

In [ ]:
df3=df3.rename(columns={"level_0":"id"})

In [ ]:
df3["combined"]=df3["Original Comment"]+". "+"Type of statement: "+df3["type"]

In [ ]:
df3.head()

,id,Original Comment,type,target,combined
0,0,Thank you so much for the excellent customer service. Thanks John,Positive,Thank you so much for the excellent customer service,Thank you so much for the excellent customer service. Thanks John. Type of statement: Positive
1,1,Impressively fast service. Thank you for correcting the damage right away.,Positive,impressively fast service; thank you for correcting the damage right away,Impressively fast service. Thank you for correcting the damage right away.. Type of statement: Positive
2,2,"I was very satisfied with the help I received from Alberto, however I still won''t receive the item I need due to refusal to deliver by the vendor even though the home depot website states that the item is able to ship to my store location. I ended up buying the product from Amazon.",Positive,very satisfied with the help I received,"I was very satisfied with the help I received from Alberto, however I still won''t receive the item I need due to refusal to deliver by the vendor even though the home depot website states that the item is able to ship to my store location. I ended up buying the product from Amazon.. Type of statement: Positive"
3,3,Appreciate the help. I also appreciate the store manager calling me on her day off to address the situation. Thank you.,Positive,appreciate the help; appreciate the store manager calling me on her day off,Appreciate the help. I also appreciate the store manager calling me on her day off to address the situation. Thank you.. Type of statement: Positive
4,4,Augustin was very pleasant to work with. He was very patient and kind in helping me to resolve the problem i had. He displayed a very good attitude toward me as a customer of Home Depot. You definitely have a _x001C_winner_x001D_ in this employee. More customer service sites could benefit from an employee like Augustin.,Positive,very pleasant to work with; he was very patient and kind in helping me resolve the problem; displayed a very good attidude; could benefit from an employee like,Augustin was very pleasant to work with. He was very patient and kind in helping me to resolve the problem i had. He displayed a very good attitude toward me as a customer of Home Depot. You definitely have a _x001C_winner_x001D_ in this employee. More customer service sites could benefit from an employee like Augustin.. Type of statement: Positive


In [ ]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['combined'] , max_length = 1024, truncation = True )

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['target'], max_length = 128, truncation = True )

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

In [ ]:
dataset_pt = df3.apply(convert_examples_to_features,axis=1)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:

# Training

from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)


In [ ]:

from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

In [ ]:
len(dataset_pt)*0.8

542.4

In [ ]:
train=dataset_pt[:543]
test=dataset_pt[543:]

In [ ]:
trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=train,eval_dataset=test)

In [ ]:

trainer.train()

You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


TrainOutput(global_step=33, training_loss=2.3060679146737764, metrics={'train_runtime': 79.2744, 'train_samples_per_second': 6.85, 'train_steps_per_second': 0.416, 'total_flos': 108442389553152.0, 'train_loss': 2.3060679146737764, 'epoch': 0.97})

In [ ]:
# Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]



def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score

In [ ]:

## Save model
model_pegasus.save_pretrained("txt_extracter_modelv1")

In [ ]:
tokenizer.save_pretrained("tokenizer1")

('tokenizer1/tokenizer_config.json',
 'tokenizer1/special_tokens_map.json',
 'tokenizer1/spiece.model',
 'tokenizer1/added_tokens.json',
 'tokenizer1/tokenizer.json')

In [ ]:
#Load

tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer1")

In [ ]:
gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}

In [ ]:
sample_text = df3["combined"][2]

reference = df3["target"][2]

pipe = pipeline("summarization", model="txt_extracter_modelv1",tokenizer=tokenizer)

In [ ]:
##
sample_text = df3["combined"][3]

sample_text="for the first time in years, this was my first bad experience at The Home Depot.but it''s all good maybe the associate was not having a good day."

reference = df3["target"][3]

print("text:")
print(sample_text)


print("\nTarget:")
print(reference)


print("\nModel Prediction:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Your max_length is set to 128, but your input_length is only 36. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=18)


text:
for the first time in years, this was my first bad experience at The Home Depot.but it''s all good maybe the associate was not having a good day.

Target:
appreciate the help; appreciate the store manager calling me on her day off

Model Prediction:
for the first time in years, this was my first bad experience at The Home Depot.<n>But it's all good maybe the associate was not having a good day.


In [ ]:
x=pipe(sample_text, **gen_kwargs)[0]["summary_text"]

Your max_length is set to 128, but your input_length is only 31. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=15)


## Testing it on the unseen data

In [ ]:
pred_unseen=pd.read_csv(f"{base_directory}/predsunseen.csv")

In [ ]:
pred_unseen=pred_unseen[["comments","pred"]]

In [ ]:
filtered_data = pred_unseen[pred_unseen['pred'] != '()']

In [ ]:
filtered_data.head()

,comments,pred
0,"for the first time in years, this was my first bad experience at The Home Depot.","('Effort', 'Negative (Other)')"
1,but it''s all good maybe the associate was not having a good day.,"('Positive',)"
2,Best service after sale. Miguel is exceptionnal.,"('Positive',)"
3,Thank you,"('Positive',)"
6,Maria was very helpful and I was able to get her the information needed.,"('Positive',)"


In [ ]:
import re
def extract_words(row):
    words = re.findall(r"'([^']*)'", row['pred'])
    return ', '.join(words)

In [ ]:
filtered_data['extracted_words'] = filtered_data.apply(extract_words,axis=1)

<ipython-input-70-90b67178ccab>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['extracted_words'] = filtered_data.apply(extract_words,axis=1)


In [ ]:
filtered_data["combined"]=filtered_data["comments"]+". "+"Type of statement: "+filtered_data["extracted_words"]

<ipython-input-73-bfde988ee095>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data["combined"]=filtered_data["comments"]+". "+"Type of statement: "+filtered_data["extracted_words"]


In [ ]:
filtered_data.head(2)

,comments,pred,extracted_words,combined
0,"for the first time in years, this was my first bad experience at The Home Depot.","('Effort', 'Negative (Other)')","Effort,Negative (Other)","for the first time in years, this was my first bad experience at The Home Depot.. Type of statement: Effort,Negative (Other)"
1,but it''s all good maybe the associate was not having a good day.,"('Positive',)",Positive,but it''s all good maybe the associate was not having a good day.. Type of statement: Positive


In [ ]:
d={}
for i,r in filtered_data.iterrows():
      sample_text=r["combined"]
      j=pipe(sample_text, **gen_kwargs)[0]["summary_text"]
      d[i]={"Comment":r["comments"],"Labels Predicted":r["pred"],"text":j}

Your max_length is set to 128, but your input_length is only 30. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=15)
Your max_length is set to 128, but your input_length is only 24. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=12)
Your max_length is set to 128, but your input_length is only 17. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=8)
Your max_length is set to 128, but your input_length is only 9. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=4)
Your ma

In [ ]:
pd.DataFrame(d).T.to_csv(f"{base_directory}/Results.csv",index=False)